In [1]:
# ИМПОРТ БИБЛИОТЕК

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde

In [2]:
# СТРОИМ БОКСПЛОТЫ ДЛЯ КОЛИЧЕСТВЕННЫХ СТОБЦОВ


def plot_boxplots(df, num_cols):
    """
    Строит box plot для каждого указанного количественного столбца с выделением выбросов.
    
    Parameters:
    df (pd.DataFrame): Исходный DataFrame
    numeric_columns (list): Список названий количественных столбцов
    """

    # Определяем количество графиков
    n = len(num_cols)
    
    # Задаем количество столбцов в сетке графиков
    cols = 3
    
    # Вычисляем количество строк в сетке графиков (округляем вверх)
    rows = (n + cols - 1) // cols

    # Создаем сетку графиков (subplots)
    # fig - весь рисунок, axes - массив осей для каждого subplot
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
    # Если график только один, преобразуем axes в массив для единообразия
    axes = axes.flatten() if n > 1 else [axes]

    # Перебираем все действительные столбцы для построения графиков
    for i, col in enumerate(num_cols):
        # Выбираем текущую ось для построения
        ax = axes[i]
        
        # Строим boxplot для текущего столбца, удаляя пропущенные значения (NaN)
        df[col].dropna().plot(kind='box', ax=ax)
        
        # Устанавливаем заголовок для текущего графика
        ax.set_title(f'Box-plot {col}')
        
        # Устанавливаем подпись для оси Y
        ax.set_ylabel('Значения')

        # Рассчитываем границы для определения выбросов
        # Q1 - первый квартиль (25-й процентиль)
        Q1 = df[col].quantile(0.25)
        
        # Q3 - третий квартиль (75-й процентиль)
        Q3 = df[col].quantile(0.75)
        
        # IQR - межквартильный размах (разница между Q3 и Q1)
        IQR = Q3 - Q1
        
        # Нижняя граница для выбросов
        lower_bound = Q1 - 1.5 * IQR
        
        # Верхняя граница для выбросов
        upper_bound = Q3 + 1.5 * IQR
        
        # Находим выбросы - значения за пределами границ
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
        
        ax.annotate(f'Выбросы: {len(outliers)}', 
                    xy=(0.03, 0.95),  # Позиция аннотации (5% от левого края, 95% от нижнего)
                    xycoords='axes fraction',  # Координаты относительно размеров оси
                    bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.5),  # Стиль рамки
                    fontsize=10)  # Размер шрифта
    
    # Показываем все графики
    plt.tight_layout()
    plt.show()

In [8]:
# СТРОИМ СКАТТЕРЫ ДЛЯ ВСЕХ СО ВСЕМИ КОЛИЧЕСТВЕННЫМИ СТОЛБЦАМИ

def plot_scatters(df, num_cols):
    """
    Строит box plot для каждого указанного количественного столбца с выделением выбросов.
    
    Parameters:
    df (pd.DataFrame): Исходный DataFrame
    numeric_columns (list): Список названий количественных столбцов
    """

    # Определяем количество графиков
    n = len(num_cols)
    
    # Задаем количество столбцов в сетке графиков
    cols = 3
    
    # Вычисляем количество строк в сетке графиков (округляем вверх)
    rows = ((n + cols - 1) // cols)

    # Создаем сетку графиков (subplots)
    # fig - весь рисунок, axes - массив осей для каждого subplot
    fig, axes = plt.subplots(rows, cols, figsize=(20, 7*rows))
    
    # Если график только один, преобразуем axes в массив для единообразия
    axes = axes.flatten() if n > 1 else [axes]

    # Перебираем все действительные столбцы для построения графиков
    arr = {}
    for i in range(n):
        arr[num_cols[i]] = []
    h = 0
    for i in range (n):
        for j in range(n):
            if num_cols[i] != num_cols[j] and num_cols[j] not in arr[num_cols[i]] and num_cols[i] not in arr[num_cols[j]]:
                
                arr[num_cols[i]].append(num_cols[j])
                ax = axes[0 + h]
                h += 1
                clean_df = df[[num_cols[i], num_cols[j]]].dropna()
                x = clean_df[num_cols[i]]
                y = clean_df[num_cols[j]]
                correlation = np.corrcoef(x, y)[0, 1]
                coefficients = np.polyfit(x, y, 1)
                reg_line = np.poly1d(coefficients)

                
                ax.scatter(x, y)
                ax.plot(x, reg_line(x), color='red', linewidth=2)
                
        
        
                # Устанавливаем заголовок для текущего графика
                ax.set_title(f'Scatter plot {num_cols[i]} and {num_cols[j]}')
        
                # Устанавливаем подпись для оси Y
                ax.set_ylabel(f'{num_cols[j]}')
                ax.set_xlabel(f'{num_cols[i]}')


        
                ax.annotate(f'Корреляция Пирсона: {correlation:.2f}', 
                            xy=(0.03, 0.95),  # Позиция аннотации (5% от левого края, 95% от нижнего)
                            xycoords='axes fraction',  # Координаты относительно размеров оси
                            bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.5),  # Стиль рамки
                            fontsize=10)  # Размер шрифта
    
    # Показываем все графики
    plt.tight_layout()
    plt.show()

In [9]:
# СТРОИМ МАТРИЦУ КОРРЕЛЯЦИЙ ДЛЯ КОЛИЧЕСТВЕННЫХ СТОЛБЦОВ

def plot_correlation_heatmap(df, numeric_columns):

    # Вычисляем корреляционную матрицу для выбранных столбцов
    corr_matrix = df[numeric_columns].corr()
    
    # Создаем тепловую карту
    plt.figure(figsize=(10, 8))
    plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    
    # Добавляем подписи осей
    plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=45)
    plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
    
    # Добавляем значения корреляции в ячейки
    for i in range(len(corr_matrix.columns)):
        for j in range(len(corr_matrix.columns)):
            plt.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}',
                    ha='center', va='center', color='black', fontsize=10)
    
    # Добавляем цветовую шкалу
    plt.colorbar(label='Корреляция')
    
    # Добавляем заголовок
    plt.title('Тепловая карта корреляционной матрицы')
    
    # Настраиваем отображение
    plt.tight_layout()
    plt.show()

In [9]:
# СТРОИМ ГИСТОГРАММЫ

def plot_histograms_with_stats(df, num_cols):
    """
    Строит гистограммы для количественных столбцов с отображением статистических характеристик.
    """

    # Создаем сетку графиков
    n = len(num_cols)
    cols = 2
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    axes = axes.flatten() if n > 1 else [axes]

    # Перебираем все действительные столбцы
    for i, col in enumerate(num_cols):
        ax = axes[i]
        clean_df = df[col].dropna()
        
        # Строим гистограмму

        # ДОБАВЛЯЕМ KDE
         # Строим гистограмму с нормализацией
        n_bins = min(30, int(np.sqrt(len(clean_df))))
        counts, bin_edges, patches = ax.hist(clean_df, bins=n_bins, alpha=0.7, edgecolor='black', density=True)
        
        # # # Добавляем KDE (ядерную оценку плотности)
        # kde = gaussian_kde(clean_df)
        # x_range = np.linspace(clean_df.min(), clean_df.max(), 1000)
        # kde_curve = kde(x_range)
        
        # # Нормализуем KDE к масштабу гистограммы
        # bin_width = bin_edges[1] - bin_edges[0]
        # hist_area = np.sum(counts * bin_width)
        # kde_curve_normalized = kde_curve * hist_area
        
        # # Строим KDE поверх гистограммы
        # ax.plot(x_range, kde_curve_normalized, color='purple', linewidth=2, label='KDE')
        
        # Вычисляем статистические показатели
        mean, median, mode = clean_df.mean(), clean_df.median(), clean_df.mode()
        variance, std_dev = clean_df.var(), clean_df.std()
        
        # Добавляем вертикальные линии
        ax.axvline(mean, color='red', linestyle='--', linewidth=2, label=f'Среднее: {mean:.2f}')
        ax.axvline(median, color='green', linestyle='--', linewidth=2, label=f'Медиана: {median:.2f}')
        
        # Обрабатываем моду
        if not mode.empty:
            first_mode = mode.iloc[0]
            ax.axvline(first_mode, color='blue', linestyle='--', linewidth=2, label=f'Мода: {first_mode:.2f}')
        
        # Добавляем статистику в текстовое поле
        stats_text = f'Среднее: {mean:.2f}\nМедиана: {median:.2f}\n'
        if not mode.empty:
            stats_text += f'Мода: {first_mode:.2f}\n'
        stats_text += f'Дисперсия: {variance:.2f}\nСтанд. отклонение: {std_dev:.2f}'
        
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
                fontsize=9, verticalalignment='top')
        
        # Настраиваем график
        ax.legend(loc='upper right')
        ax.set_title(f'Гистограмма для {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Частота')
        ax.grid(True, linestyle='--', alpha=0.7)

    
    plt.tight_layout()
    plt.show()

# Пример использования:
# plot_histograms_with_stats(df, ['age', 'income', 'score'])
